In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import joblib
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import bayesian_search_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.preprocessing import RollingFeatures
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
import os

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series
THIS_DIR = Path("C:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast\\skforecast\\model_selection\\tests")
THIS_DIR = Path(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests")
series_item_sales = pd.read_parquet(THIS_DIR/'fixture_multi_series_items_sales.parquet')
series_item_sales = series_item_sales.asfreq('D')
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [22]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
# from skopt.space import Real
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection._search import bayesian_search_forecaster
from skforecast.model_selection._split import TimeSeriesFold

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog

In [23]:
import os
import re
import sys
import pytest
from pytest import approx
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._search import _bayesian_search_optuna_multiseries
from skforecast.model_selection._search import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection._search import _evaluate_grid_hyperparameters
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
from functools import partialmethod

import re
import pytest
import os
import sys
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection._search import _bayesian_search_optuna
from skforecast.model_selection._split import TimeSeriesFold
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
from functools import partialmethod
from sklearn.linear_model import LinearRegression
from skforecast.model_selection._validation import _backtesting_forecaster

In [27]:
window_features = RollingFeatures(
    stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes = 3,
)
forecaster = ForecasterDirect(
    regressor=LinearRegression(), steps=4, lags=3, window_features=window_features
)

n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
        steps                 = 4,
        initial_train_size    = len(y_train),
        window_size           = None,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )
metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster = forecaster,
                                    y          = y,
                                    exog       = exog,
                                    cv         = cv,
                                    metric     = 'mean_squared_error',
                                    verbose    = False
                                )

  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
metric.to_numpy()

array([[0.11337407]])

In [29]:
backtest_predictions.to_numpy().flatten()

array([0.31478125, 0.45940314, 0.85744121, 0.67976412, 0.38085263,
       0.56357911, 0.35567212, 0.4493073 , 0.35493806, 0.53589892,
       0.73060039, 0.60025524])

In [9]:
results.to_dict(orient='list')

{'lags': [array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2])],
 'lags_label': [array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2])],
 'params': [{'alpha': 1},
  {'alpha': 1},
  {'alpha': 0.1},
  {'alpha': 0.1},
  {'alpha': 0.01},
  {'alpha': 0.01}],
 'mean_squared_error': [0.07424336814199613,
  0.07576112822191301,
  0.07677225532446876,
  0.08046370393440734,
  0.08411584414110274,
  0.08911485459058603],
 'alpha': [1.0, 1.0, 0.1, 0.1, 0.01, 0.01]}

In [30]:
import numpy as np
import pandas as pd
from skforecast.sarimax import Sarimax
from skforecast.recursive import ForecasterSarimax
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.model_selection._search import random_search_sarimax

# Fixtures
from skforecast.recursive.tests.tests_forecaster_sarimax.fixtures_forecaster_sarimax import y_datetime


In [32]:
forecaster = ForecasterSarimax(
                     regressor = Sarimax(order=(3, 2, 0), maxiter=1000, method='cg', disp=False)
                 )
    
cv = TimeSeriesFold(
            steps                 = 3,
            initial_train_size    = len(y_datetime) - 12,
            refit                 = False,
            fixed_train_size      = False,
            gap                   = 0,
            allow_incomplete_fold = True
        )
cv = OneStepAheadFold(
            initial_train_size    = len(y_datetime) - 12,
            return_all_indexes    = False,
        )

param_distributions = {'order': [(2, 2, 0), (3, 2, 0)]}

results = random_search_sarimax(
                forecaster          = forecaster,
                y                   = y_datetime,
                cv                  = cv,
                param_distributions = param_distributions,
                metric              = 'mean_absolute_error',
                n_iter              = 2,
                random_state        = 123,
                return_best         = False,
                verbose             = False
            )

Number of models compared: 2.


params grid:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: `cv` must be a TimeSeriesFold object. Got OneStepAheadFold.

In [34]:
cv = TimeSeriesFold(
            steps                 = 3,
            initial_train_size    = 10,
            refit                 = False,
            fixed_train_size      = False,
            gap                   = 0,
            allow_incomplete_fold = True
        )

In [36]:
current_params = vars(cv)
current_params

params = {'steps': 53}

In [37]:
{'cv_name': type(cv).__name__, **current_params, **params}

{'cv_name': 'TimeSeriesFold',
 'steps': 53,
 'initial_train_size': 10,
 'window_size': None,
 'differentiation': None,
 'refit': False,
 'fixed_train_size': False,
 'gap': 0,
 'skip_folds': None,
 'allow_incomplete_fold': True,
 'return_all_indexes': False,
 'verbose': True}

In [51]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = TimeSeriesFold(
        steps                 = 10,
        initial_train_size    = 70,
        window_size           = None,
        differentiation       = None,
        refit                 = False,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
        verbose               = False
    )
folds = cv.split(X=y, as_pandas=True)
folds

c:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\_split.py:791: UserWarning: Last window cannot be calculated because `window_size` is None.
  externally_fitted = False


,fold,train_start,train_end,last_window_start,last_window_end,test_start,test_end,test_start_with_gap,test_end_with_gap,fit_forecaster
0,0,0,70,None,None,70,80,70,80,True
1,1,0,70,None,None,80,90,80,90,False
2,2,0,70,None,None,90,100,90,100,False


In [60]:
expected = pd.DataFrame(
        {'fold': [0, 1, 2],
         'train_start': [0, 0, 0],
         'train_end': [70, 70, 70],
         'last_window_start': [67, 77, 87],
         'last_window_end': [70, 80, 90],
         'test_start': [70, 80, 90],
         'test_end': [80, 90, 100],
         'test_start_with_gap': [70, 80, 90],
         'test_end_with_gap': [80, 90, 100],
         'fit_forecaster': [True, False, False]}
    )
expected['last_window_start'] = [None, None, None]
expected

,fold,train_start,train_end,last_window_start,last_window_end,test_start,test_end,test_start_with_gap,test_end_with_gap,fit_forecaster
0,0,0,70,None,70,70,80,70,80,True
1,1,0,70,None,80,80,90,80,90,False
2,2,0,70,None,90,90,100,90,100,False


In [63]:
expected.dtypes

fold                    int64
train_start             int64
train_end               int64
last_window_start      object
last_window_end         int64
test_start              int64
test_end                int64
test_start_with_gap     int64
test_end_with_gap       int64
fit_forecaster           bool
dtype: object

In [62]:
folds.dtypes

fold                    int64
train_start             int64
train_end               int64
last_window_start      object
last_window_end        object
test_start              int64
test_end                int64
test_start_with_gap     int64
test_end_with_gap       int64
fit_forecaster           bool
dtype: object

In [44]:
folds.to_dict(orient='list')

{'fold': [0, 1, 2],
 'train_start': [0, 0, 0],
 'train_end': [70, 70, 70],
 'last_window_start': [67, 77, 87],
 'last_window_end': [70, 80, 90],
 'test_start': [70, 80, 90],
 'test_end': [80, 90, 100],
 'test_start_with_gap': [70, 80, 90],
 'test_end_with_gap': [80, 90, 100],
 'fit_forecaster': [True, False, False]}

In [56]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = TimeSeriesFold(
        steps                 = 10,
        initial_train_size    = 70,
        window_size           = None,
        differentiation       = None,
        refit                 = False,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = True,
        verbose               = False
    )
folds_2 = cv.split(X=y, as_pandas=True)
folds_2

c:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\_split.py:791: UserWarning: Last window cannot be calculated because `window_size` is None.
  externally_fitted = False


,fold,train_index,last_window_index,test_index,test_index_with_gap,fit_forecaster
0,0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[None, None]","(70, 71, 72, 73, 74, 75, 76, 77, 78, 79)","(70, 71, 72, 73, 74, 75, 76, 77, 78, 79)",True
1,1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[None, None]","(80, 81, 82, 83, 84, 85, 86, 87, 88, 89)","(80, 81, 82, 83, 84, 85, 86, 87, 88, 89)",False
2,2,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[None, None]","(90, 91, 92, 93, 94, 95, 96, 97, 98, 99)","(90, 91, 92, 93, 94, 95, 96, 97, 98, 99)",False


In [58]:
folds_2.dtypes

fold                    int64
train_index            object
last_window_index      object
test_index             object
test_index_with_gap    object
fit_forecaster           bool
dtype: object

In [57]:
folds_2.to_dict(orient='list')

{'fold': [0, 1, 2],
 'train_index': [range(0, 70), range(0, 70), range(0, 70)],
 'last_window_index': [[None, None], [None, None], [None, None]],
 'test_index': [range(70, 80), range(80, 90), range(90, 100)],
 'test_index_with_gap': [range(70, 80), range(80, 90), range(90, 100)],
 'fit_forecaster': [True, False, False]}

In [64]:
from skforecast.preprocessing import QuantileBinner

In [70]:
X = np.arange(1000)
binner = QuantileBinner(
    n_bins=10,
    method='linear',
    subsample=10,
    dtype=np.float64,
    random_state=789654,
)

binner.fit(X.reshape(-1, 1))


In [71]:
binner.bin_edges_

array([ 21. , 159.6, 283.8, 388.7, 429.8, 578. , 725. , 731.6, 734.2,
       739.4, 743. ])

In [72]:
binner.n_bins_

10

In [73]:
binner.intervals_

{0.0: (21.0, 159.6),
 1.0: (159.6, 283.8),
 2.0: (283.8, 388.7),
 3.0: (388.7, 429.8),
 4.0: (429.8, 578.0),
 5.0: (578.0, 725.0),
 6.0: (725.0, 731.6),
 7.0: (731.6, 734.2),
 8.0: (734.2, 739.4),
 9.0: (739.4, 743.0)}